# Exporting data to Xdmf format

Author(s): Jukka Aho <jukka.aho@kapsi.fi>

Python example how to export data to Xdmf format using Python

In [1]:
import sys
import os
sys.path.append(os.path.expanduser('~/opt/xdmf/lib/python'))

In [2]:
from Xdmf import *

## Export mesh

In [3]:
grid = XdmfUnstructuredGrid.New()
grid.setTime(XdmfTime.New(123))
geometry = XdmfGeometry.New()
topology = XdmfTopology.New()
geometry.setType(XdmfGeometryType.XYZ())
topology.setType(XdmfTopologyType.Mixed())

In [4]:
import numpy as np
# one linear tetra
nodes = np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
nodes

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

It needs to be in 1d list:

In [6]:
list(nodes.flatten())

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [7]:
geometry.insertAsFloat32(0, list(nodes.flatten()))

Topology; connect nodes 1 - 4 to form tetra

List is here:

http://public.kiteware.com/pipermail/2013-July/028859.html

In [9]:
topology.insertAsInt32(0, [0x6, 1, 2, 3, 4]) # Not sure starts this from 0 or 1

**update**, node numbering starts from 0.

In [10]:
grid.setGeometry(geometry)
grid.setTopology(topology)

## Export fields

Give scalar value to cells (i.e. elements)

In [11]:
temperature = XdmfAttribute.New()
temperature.setType(XdmfAttributeType.Scalar())
temperature.setCenter(XdmfAttributeCenter.Cell())
temperature.setName("Temperature field")
temperature.insertAsInt32(0, [56])
grid.insert(temperature)

Give vector values to nodes

In [13]:
density = XdmfAttribute.New()
density.setType(XdmfAttributeType.Vector())
density.setCenter(XdmfAttributeCenter.Node())
density.setName("Density")
density.insertAsFloat32(0, [1.3, 2.4, 4.5, 6.7])
grid.insert(density)

## Write to file

I think this is not mandatory but create temporal collection anyway:

In [14]:
coll = XdmfGridCollection.New()
coll.setType(XdmfGridCollectionType.Temporal())
coll.insert(grid)

Create domain and write to disk

In [18]:
domain = XdmfDomain.New()
domain.insert(coll)
writer = XdmfWriter.New("testdata.xmf")
domain.accept(writer)

In [20]:
!ls testdata*

testdata.h5  testdata.xmf


In [21]:
!cat testdata.xmf

<?xml version="1.0" encoding="utf-8"?>
<Xdmf xmlns:xi="http://www.w3.org/2001/XInclude" Version="2.1">
  <Domain>
    <Grid CollectionType="Temporal" GridType="Collection" Name="Collection">
      <Geometry Type="None"/>
      <Topology Dimensions="0" Type="NoTopology"/>
      <Grid Name="Grid">
        <Time Value="123"/>
        <Geometry Type="XYZ">
          <DataItem DataType="Float" Dimensions="12" Format="XML" Precision="4">0 0 0 1 0 0 0 1 0 0 0 1</DataItem>
        </Geometry>
        <Topology Dimensions="1" Type="Mixed">
          <DataItem DataType="Int" Dimensions="5" Format="XML" Precision="4">6 1 2 3 4</DataItem>
        </Topology>
        <Attribute Center="Cell" Name="Temperature field" Type="Scalar">
          <DataItem DataType="Int" Dimensions="1" Format="XML" Precision="4">56</DataItem>
        </Attribute>
        <Attribute Center="Node" Name="Density" Type="Vector">
          <DataItem DataType="Float" Dimensions="4" Format="XML" Precision="4">1.3 2.4000001 4.5 

This time Xdmf decided to write results directly to the xml file. If the model is bigger the correct format is

    <?xml version="1.0" encoding="utf-8"?>
    <Xdmf xmlns:xi="http://www.w3.org/2001/XInclude" Version="2.1">
      <Domain>
        <Grid CollectionType="Temporal" GridType="Collection" Name="Collection">
          <Geometry Type="None"/>
          <Topology Dimensions="0" Type="NoTopology"/>
          <Grid Name="Grid">
            <Time Value="0"/>
            <Geometry Type="XYZ">
              <DataItem DataType="Float" Dimensions="1260999" Format="HDF" Precision="4">model.h5:Data16</DataItem>
            </Geometry>
            <Topology Dimensions="292001" Type="Mixed">
              <DataItem DataType="Int" Dimensions="3046831" Format="HDF" Precision="4">model.h5:Data17</DataItem>
            </Topology>
            <Attribute Center="Cell" Name="Temperature field" Type="Scalar">
              <DataItem DataType="Int" Dimensions="292001" Format="HDF" Precision="4">model.h5:Data18</DataItem>
            </Attribute>
            <Attribute Center="Node" Name="Density" Type="Vector">
              <DataItem DataType="Float" Dimensions="292001" Format="HDF" Precision="4">model.h5:Data19</DataItem>
            </Attribute>
          </Grid>
        </Grid>
      </Domain>
    </Xdmf>


Structure of h5 file


    File(filename=model.h5, title='', mode='r+', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))                                                                                                                                   
    / (RootGroup) ''       
    /Data0 (EArray(1125135,)) ''
      atom := Float32Atom(shape=(), dflt=0.0)
      maindim := 0        
      flavor := 'numpy'         
      byteorder := 'little'      
      chunkshape := (1000,)     
    /Data1 (EArray(2680009,)) ''
      atom := Int32Atom(shape=(), dflt=0)
      maindim := 0        
      flavor := 'numpy'         
      byteorder := 'little'      
      chunkshape := (1000,)        
    ...


    In [7]: d.root.Data0[:]
    Out[7]:
    array([ 302.10440063,  222.19999695,   20.38028908, ..., -230.        ,
              0.        ,    7.33333015], dtype=float32)
